##20600 - Deep Learning for Computer Vision
##Bocconi Univeristy
##Logo Detection Project

##Group 4:
Zhina Aghamohammadi, Rodolfo Melo, Samuele Rabino, Andrea Raminelli, Matteo Salvietti, Leonardo Yang



### 1. Configure environment / download Object Detection API github

In [ ]:
import os
import pathlib
# Clone the tensorflow models repository
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 3135, done.
remote: Counting objects: 100% (3135/3135), done.
remote: Compressing objects: 100% (2648/2648), done.
remote: Total 3135 (delta 809), reused 1343 (delta 443), pack-reused 0
Receiving objects: 100% (3135/3135), 33.35 MiB | 25.87 MiB/s, done.
Resolving deltas: 100% (809/809), done.


In [ ]:
%%shell
python -m pip install --upgrade pip
sudo apt install -y protobuf-compiler
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

     |████████████████████████████████| 1.7 MB 5.2 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
Reading package lists... Done
Building dependency tree       
Reading state information... Done
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
Processing /content/models/research
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 9.8 MB 8.6 MB/s            
     |████████████████████████████████| 352 kB 41.0 MB/s            
     |████████████████████████████████| 1.8 MB 17.5 MB/s            
     |████████████████████████████████| 22.7 MB 2.1 MB/s             
     |████████████████████████████████| 1.3 MB 28.9 MB/s            
     |████████████████████████████████| 1.1 MB 32.5 MB/s            
     |████████████████████████████████| 

#2. Importing libraries and define functions

In [ ]:
import io
import os
import scipy.misc
import numpy as np
import six
import time
import pathlib
import pandas as pd
from os import listdir
from os.path import isfile, join
import cv2

from six import BytesIO

import matplotlib
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont

import tensorflow as tf
from object_detection.utils import visualization_utils as viz_utils
from PIL import Image
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Function img to np
def load_image_into_numpy_array(path):

    img_data = tf.io.gfile.GFile(path, 'rb').read()
    image = Image.open(BytesIO(img_data))
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

#Function to compute IOU
def bb_intersection_over_union(boxA, boxB):
    # determine the (x, y) coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    # compute the area of intersection rectangle
    interArea = max(0, xB - xA) * max(0, yB - yA)
    # compute the area of both the prediction and ground-truth rectangles
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

# Func int to text class:
def class_text_to_int(row_label):
    if row_label == 'Nike':
        return 1
    elif row_label == 'Adidas':
        return 2
    elif row_label == 'Starbucks':
        return 3
    elif row_label == 'Apple Inc.':
        return 4
    elif row_label == 'NFL':
        return 5
    elif row_label == 'Mercedes-Benz':
        return 6
    elif row_label == 'Under Armour':
        return 7
    elif row_label == 'Coca-Cola':
        return 8
    elif row_label == 'Hard Rock Cafe':
        return 9
    elif row_label == 'Puma':
        return 10
    elif row_label == 'The North Face':
        return 11
    elif row_label == 'Toyota':
        return 12
    elif row_label == 'Chanel':
        return 13
    elif row_label == 'Pepsi':
        return 14
    else:
        None

#3. Load a pretrained model and evaluate it on the test data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load a saved model

tf.keras.backend.clear_session()
detect_fn = tf.saved_model.load('/content/drive/MyDrive/DLCV_group/model_centernet_14logos/saved_model')

In [ ]:
image_dir = '/content/drive/MyDrive/DLCV_group/data/test'

image_path_list = [f for f in listdir(image_dir) if isfile(join(image_dir, f))]

prediction = pd.DataFrame(index= range(len(image_path_list)),columns=['filename', 'yminB', 'xminB', 'ymaxB','xmaxB','classB','score'])

#it takes up to 10h in colab, we run it on the virtual machine
for i in range(len(image_path_list)):
    image_path = os.path.join(image_dir, image_path_list[i])
    image_np = load_image_into_numpy_array(image_path)
    input_tensor = np.expand_dims(image_np, 0)
    detections = detect_fn(input_tensor)
    prediction['filename'][i]=image_path_list[i]
    prediction['yminB'][i]=detections['detection_boxes'][0][0].numpy()[0]
    prediction['xminB'][i]=detections['detection_boxes'][0][0].numpy()[1]
    prediction['ymaxB'][i]=detections['detection_boxes'][0][0].numpy()[2]
    prediction['xmaxB'][i]=detections['detection_boxes'][0][0].numpy()[3]
    prediction['classB'][i]=detections['detection_classes'][0][0].numpy().astype(np.int32)
    prediction['score'][i]=detections['detection_scores'][0][0]
    if i % 100 == 0:
        print('{} Pictures processed'.format(i))
    
prediction.to_csv('temp_result.csv', index=False)

0 Pictures processed
100 Pictures processed


Normalize images because we got wrong annotations. Then apply our model on the test data and calculate the IOU.


In [ ]:
# Upload temp_result.csv
prediction = pd.read_csv('temp_result.csv')
prediction['score'] = prediction['score'].apply(lambda x: float(x[10:12]))
prediction['classB'] = prediction['classB'].apply(class_int_to_text)
prediction = prediction.loc[:,['filename','xminB','yminB','xmaxB','ymaxB','classB','score']]

# Upload test set and normalizing the box dimension
test_set = pd.read_csv('/content/drive/MyDrive/DLCV_group/data/test.csv')
test_set = test_set.rename(columns={'photo_filename':'filename'})

# Get width and height of every image
image_dir = '/content/drive/MyDrive/DLCV_groupdata/test'
for i in range(len(test_set)):
    image_path = os.path.join(image_dir, test_set['filename'][i])
    im = Image.open(image_path)
    w, h = im.size
    test_set['width'][i]=w
    test_set['height'][i]=h

# Merging
test_prediction = pd.merge(test_set,prediction,on='filename')

# Get the full box prediction
test_prediction['xminB'] = test_prediction['xminB']*test_prediction['width']
test_prediction['xmaxB'] = test_prediction['xmaxB']*test_prediction['width']
test_prediction['yminB'] = test_prediction['yminB']*test_prediction['height']
test_prediction['ymaxB'] = test_prediction['ymaxB']*test_prediction['height']

# Calculating IOU
test_prediction['IOU'] = np.nan
for i in range(len(test_prediction)):
    boxA = [test_prediction['xmin'][i],test_prediction['ymin'][i],test_prediction['xmax'][i],test_prediction['ymax'][i]]
    boxB = [test_prediction['xminB'][i],test_prediction['yminB'][i],test_prediction['xmaxB'][i],test_prediction['ymaxB'][i]]
    test_prediction['IOU'][i] = bb_intersection_over_union(boxA, boxB)
    

test_prediction.to_csv('final_result.csv',index=False)

#4. Results
Average perfomances of our model for each brand

In [ ]:
# Result mean IOU by class
test_prediction= pd.read_csv('/content/drive/MyDrive/DLCV_group/final_result.csv')
result_iou = test_prediction.groupby('class').mean()[['IOU']].reset_index().sort_values(by=['IOU'],ascending=False).reset_index(drop=True)
result_iou['IOU'] = np.around(result_iou['IOU'], decimals=3)
result_iou.to_csv('final_results_agg_class.csv',index=False)
result_iou

,class,IOU
0,The North Face,0.920
1,Starbucks,0.919
2,Toyota,0.908
3,Mercedes-Benz,0.905
4,Under Armour,0.899
5,Adidas,0.898
6,Apple Inc.,0.894
7,Hard Rock Cafe,0.894
8,Puma,0.873
9,NFL,0.868
